In [ ]:
import os.path
import datetime
import warnings
warnings.filterwarnings('ignore')


import ipywidgets

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (interactive but deprecated) or "%matplotlib inline" (static plots)
%matplotlib widget
import matplotlib.pyplot

import numpy as np
import netCDF4
from matplotlib import pyplot
from matplotlib import dates

setup_dir = "../../eat/tests/nns_annual_igotm"

In [ ]:
nc = netCDF4.Dataset(os.path.join(setup_dir, 'result.nc'))

z = nc['z'][:, :, 0, 0]
nctime = nc['time']
time = netCDF4.num2date(nctime[:], units=nctime.units)
time = dates.date2num(time)
time = np.broadcast_to(time[:, np.newaxis], z.shape)

matplotlib.pyplot.ioff()
fig, ax = matplotlib.pyplot.subplots()
pc = ax.pcolormesh(time, z, nc['temp'][:, :, 0, 0])
cb = fig.colorbar(pc, ax=ax)
ax.set_xlabel('time (d)')
ax.set_ylabel('height (m)')

def update(v):
    pc.set_array(v[:, :, 0, 0].ravel())
    cb.set_label('%s (%s)' % (v.long_name, v.units))
    ax.set_title(v.long_name)
    # ax.relim()
    # ax.autoscale()
    fig.canvas.draw()
available = [(v.long_name, v) for v in nc.variables.values() if v.dimensions == ('time', 'z', 'lat', 'lon')]
dropdown = ipywidgets.interactive(update, v=available)
display(dropdown)
fig